<a href="https://colab.research.google.com/github/senowmon/Fine-tuning-wav2vec2-ASR-Indonesia/blob/main/Fine_tuning_Wav2Vec2_for_Indonesia_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

huggingface [Source Code](https://huggingface.co/juanhebert/wav2vec2-indonesia) https://huggingface.co/juanhebert/wav2vec2-indonesia

# **Fine-tuning Wav2Vec2 for Indonesia ASR with  Transformers**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Feb 24 07:11:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
%%capture
!apt install git-lfs

In [ ]:
from datasets import load_dataset

dataset = load_dataset("common_voice", "id")

Downloading:   0%|          | 0.00/4.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/id/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2130
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1844
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1835
    })
    other: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 6782
    })
    invalidated: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 470
    })
})

In [ ]:
dataset = dataset.remove_columns(["client_id", "path", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"].remove_columns(["audio"]), num_examples=10)

,sentence
0,Aku tidak percaya padamu.
1,Semuanya seribu enam ratus yen.
2,Tom merasa ini bukan ide yang baik.
3,Ruangan ini akan segera menjadi lebih hangat.
4,Menjadi tunangan bukanlah menjadi suami.
5,Mereka menghadirinya.
6,"Tamu duduk beberapa menit, tetapi melihat semuanya."
7,Langit menjadi sedikit mendung.
8,Apakah Tom memintamu untuk menulis ini?
9,Saya hanya bergurau.


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
dataset = dataset.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
show_random_elements(dataset["train"].remove_columns(["audio"]))

,sentence
0,tas yang besar praktis
1,halo
2,buku harry potter apa yang paling anda suka
3,saya keluar rumah pukul sembilan
4,menyakiti orang lain itu tidak baik
5,adik perempuan saya matanya besar dan hidungnya mancung
6,saya tidak suka menggunakan googleterjemahan
7,semua bajunya dibuat oleh dia sendiri
8,ambil jalan di sebelah kanan
9,pak guru ini ibu saya


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 25,
 'a': 22,
 'b': 12,
 'c': 13,
 'd': 1,
 'e': 6,
 'f': 9,
 'g': 14,
 'h': 23,
 'i': 18,
 'j': 7,
 'k': 8,
 'l': 17,
 'm': 24,
 'n': 16,
 'o': 20,
 'p': 3,
 'r': 4,
 's': 26,
 't': 2,
 'u': 19,
 'v': 21,
 'w': 5,
 'x': 0,
 'y': 10,
 'z': 11,
 '！': 15}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-indonesia"

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/juanhebert/wav2vec2-indonesia into local empty directory.


Download file pytorch_model.bin:   0%|          | 595/360M [00:00<?, ?B/s]

Download file runs/Feb24_05-40-08_65d9269b3ca4/events.out.tfevents.1645681310.65d9269b3ca4.2420.0:  78%|######…

Download file training_args.bin: 100%|##########| 2.73k/2.73k [00:00<?, ?B/s]

Download file runs/Feb24_05-40-08_65d9269b3ca4/1645681310.3543236/events.out.tfevents.1645681310.65d9269b3ca4.…

...

Clean file training_args.bin:  37%|###6      | 1.00k/2.73k [00:00<?, ?B/s]

Clean file runs/Feb24_05-40-08_65d9269b3ca4/1645681310.3543236/events.out.tfevents.1645681310.65d9269b3ca4.242…

Clean file pytorch_model.bin:   0%|          | 1.00k/360M [00:00<?, ?B/s]

To https://huggingface.co/juanhebert/wav2vec2-indonesia
   f91faf3..fbbc04a  main -> main



'https://huggingface.co/juanhebert/wav2vec2-indonesia/commit/fbbc04a78210fab7d475f1bf55ac6d209a87cbe4'

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=48000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
dataset["train"][0]["audio"]

{'array': array([ 0.        ,  0.        ,  0.        , ...,  0.00033009,
        -0.00013936, -0.00057721], dtype=float32),
 'path': 'cv-corpus-6.1-2020-12-11/id/clips/common_voice_id_19059295.mp3',
 'sampling_rate': 48000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset["train"]))

print(dataset["train"][rand_int]["sentence"])
ipd.Audio(data=np.asarray(dataset["train"][rand_int]["audio"]["array"]), autoplay=True, rate=48000)

dia sangat puas dengan mobil barunya 


In [ ]:
rand_int = random.randint(0, len(dataset["train"]))

print("Target text:", dataset["train"][rand_int]["sentence"])
print("Input array shape:", np.asarray(dataset["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", dataset["train"][rand_int]["audio"]["sampling_rate"])

Target text: mereka belum pulang ke rumah 
Input array shape: (120960,)
Sampling rate: 48000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

In [ ]:
max_input_length_in_sec = 4.0
dataset["train"] = dataset["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.codevectors', 'project_q.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=5,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=100,
  push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

/content/wav2vec2-indonesia is already a clone of https://huggingface.co/juanhebert/wav2vec2-indonesia. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp fp16 backend


### Training

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 1470
  Num Epochs = 2
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 588
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss,Wer
200,2.874400,3.030106,1.000000
400,2.868000,3.072660,1.000000


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 1844
  Batch size = 8
Saving model checkpoint to wav2vec2-indonesia/checkpoint-200
Configuration saved in wav2vec2-indonesia/checkpoint-200/config.json
Model weights saved in wav2vec2-indonesia/checkpoint-200/pytorch_model.bin
Configuration saved in wav2vec2-indonesia/checkpoint-200/preprocessor_config.json
Configuration saved in wav2vec2-indonesia/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use 

TrainOutput(global_step=588, training_loss=2.872013740799054, metrics={'train_runtime': 3227.1258, 'train_samples_per_second': 0.911, 'train_steps_per_second': 0.182, 'total_flos': 2.526549372997632e+17, 'train_loss': 2.872013740799054, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-indonesia
Configuration saved in wav2vec2-indonesia/config.json
Model weights saved in wav2vec2-indonesia/pytorch_model.bin
Configuration saved in wav2vec2-indonesia/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.38k/360M [00:00<?, ?B/s]

Upload file runs/Feb24_11-13-29_3b3717bfbddc/events.out.tfevents.1645701307.3b3717bfbddc.84.4:  60%|######    …

To https://huggingface.co/juanhebert/wav2vec2-indonesia
   932efb0..aae2c68  main -> main

Dropping the following result as it does not have all the necessary field:
{}
To https://huggingface.co/juanhebert/wav2vec2-indonesia
   aae2c68..153beb2  main -> main



'https://huggingface.co/juanhebert/wav2vec2-indonesia/commit/aae2c68ad8bcb9a4f979e6c10665f7585fd13193'

### Evaluate

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("juanhebert/wav2vec2-indonesia")

loading feature extractor configuration file https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/5823cb931198bbc0265e2a5b504480a20f16fe3e7f241759c182285facf3792c.6e1ae58b60b48dba03f97a0c899be319c7f3e024ab623a66a7b684894f775c50
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 48000
}

loading file https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/b9f32cf58eeaecb041e362ca8f4faf4609fb2576eea673616fdba03d26a3bdfa.3e4026a0e3b79a55e6180c4eedb5a1380ffd88b6bb2dfff4a8ce9c04e5ae3f25
loading file https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformer

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("juanhebert/wav2vec2-indonesia")

https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpas1s8mi3


Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

storing https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/db4487bda549b81a2834b2325b704378e9e0ffc678259084749235f94d71201d.8107a4eaf108626f6917281c5410faa1c0a8be269ef34ac60eef1cb2724d4933
creating metadata file for /root/.cache/huggingface/transformers/db4487bda549b81a2834b2325b704378e9e0ffc678259084749235f94d71201d.8107a4eaf108626f6917281c5410faa1c0a8be269ef34ac60eef1cb2724d4933
loading configuration file https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/db4487bda549b81a2834b2325b704378e9e0ffc678259084749235f94d71201d.8107a4eaf108626f6917281c5410faa1c0a8be269ef34ac60eef1cb2724d4933
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_siz

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

storing https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/7dce3be585d91bb62a9b0cc84b8f18f7beb4ebc9736bf3a39bd476d349d8213b.782e9ce0fb9b3f091b7d6dd37be5b129aab5dcc0b4aaab54e66b4a6031d62d1b
creating metadata file for /root/.cache/huggingface/transformers/7dce3be585d91bb62a9b0cc84b8f18f7beb4ebc9736bf3a39bd476d349d8213b.782e9ce0fb9b3f091b7d6dd37be5b129aab5dcc0b4aaab54e66b4a6031d62d1b
loading weights file https://huggingface.co/juanhebert/wav2vec2-indonesia/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/7dce3be585d91bb62a9b0cc84b8f18f7beb4ebc9736bf3a39bd476d349d8213b.782e9ce0fb9b3f091b7d6dd37be5b129aab5dcc0b4aaab54e66b4a6031d62d1b
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at juanhebert/wav2vec2-indonesia.
If your task is similar to the task the model of the checkpoi

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], 
                                device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["sentence"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = dataset["test"].map(map_to_result,remove_columns=dataset["test"].column_names)

0ex [00:00, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["sentence"])))

Test WER: 1.000


In [ ]:
show_random_elements(results)

,pred_str,sentence
0,,apakah anda butuh tempat tinggal
1,,kalau dari sini pergi sedikit ke barat ada laut
2,,di kamar mandi tidak ada sabun
3,,terima kasih untuk pertolongan anda
4,,saya mengembalikan buku ke perpustakaan
5,,biasanya kami memanggil dia toshi
6,,minggu depan kakak lakilaki pak lee datang bermain ke jepang
7,,film ini tidak begitu bagus
8,,hai pak tanaka selamat malam
9,,apakah kau melihat ponselku ada di meja


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(dataset["test"][:1]["input_values"], 
                              device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA